### Temperature plot

In [5]:
import os
import glob
import rasterio
import numpy as np
import imageio
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

def kelvin_to_celsius(kelvin_array):
    """Convert temperature from Kelvin to Celsius, ignoring values < 250 K."""
    celsius_array = np.where(kelvin_array < 250, np.nan, kelvin_array - 273.15)
    return celsius_array

def extract_year(file_name):
    """Extract year from the file name."""
    return file_name.split('_')[-1].split('.')[0]

# Folder paths
folder_path = "C:/Users/carsk/OneDrive - KU Leuven/Personal_projects/Temperature_map_EC/Amazonia"
output_folder = os.path.join(folder_path, "temp_images")
os.makedirs(output_folder, exist_ok=True)

# File patterns
pattern = os.path.join(folder_path, '*.tif')
file_paths = sorted(glob.glob(pattern))

temp_images = []

for file_path in file_paths:
    year = extract_year(os.path.basename(file_path))
    with rasterio.open(file_path) as src:
        # Convert raster to numpy array and apply temperature conversion
        array = src.read(1)
        array_celsius = kelvin_to_celsius(array)
        array_celsius[array_celsius < -273.15] = np.nan  # Filter out no data values
        
        # Plotting with Cartopy
        fig, ax = plt.subplots(figsize=(13.03, 8), subplot_kw={'projection': ccrs.PlateCarree()})
        # Main title and subtitle with a newline character
        title_with_subtitle = f"Temperatura Anual Promedio - {year}\nREGIÓN AMAZÓNICA"

        ax.set_title(title_with_subtitle, fontsize=14, pad=20, loc='center')

        
        # Add global political borders and coastlines for context
        ax.add_feature(cfeature.BORDERS, linestyle=':')
        ax.add_feature(cfeature.COASTLINE)

        # Adding the grid
        ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
        
        # Plotting the temperature data
        mappable = ax.imshow(array_celsius, cmap='coolwarm', vmin=0, vmax=30, transform=ccrs.PlateCarree(), extent=[src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top])
        plt.colorbar(mappable, ax=ax, shrink=0.5, aspect=5, label='Temperatura (°C)')
        
        plt.axis('off')
        
        temp_image_path = os.path.join(output_folder, f"{year}.png")
        plt.savefig(temp_image_path, bbox_inches='tight', pad_inches=0)
        plt.close()
        
        temp_images.append(temp_image_path)

# Create a GIF
gif_path = os.path.join(folder_path, "annual_temperature.gif")
with imageio.get_writer(gif_path, mode='I', duration=0.5) as writer:
    for temp_image in temp_images:
        image = imageio.imread(temp_image)
        writer.append_data(image)

# Optionally, remove the temporary images
"""for temp_image in temp_images:
    os.remove(temp_image)
"""
print(f"GIF created at {gif_path}")


C:\Users\carsk\AppData\Local\Temp\ipykernel_36100\580658330.py:69: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(temp_image)


GIF created at C:/Users/carsk/OneDrive - KU Leuven/Personal_projects/Temperature_map_EC/Amazonia\annual_temperature.gif


### Time series

In [17]:
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from osgeo import gdal
import numpy as np
import imageio

# Setup seaborn for aesthetics
sns.set_theme(style="whitegrid")
sns.set_context("notebook", font_scale=0.8)

def kelvin_to_celsius(kelvin_array):
    """Convert temperature from Kelvin to Celsius, ignoring values < 250 K."""
    celsius_array = np.where(kelvin_array < 250, np.nan, kelvin_array - 273.15)
    return celsius_array

# Extract year from the file name
def extract_year(file_name):
    return int(file_name.split('_')[-1].split('.')[0])

# Specify your folder path containing TIFF files
folder_path = "C:\\Users\\carsk\\OneDrive - KU Leuven\\Personal_projects\\Temperature_map_EC\\costa"
# Pattern to match TIFF files
pattern = os.path.join(folder_path, '*.tif')
file_paths = sorted(glob.glob(pattern), key=lambda x: extract_year(x))

# Initialize lists to hold years and their mean temperatures
years = []
mean_temperatures = []

for file_path in file_paths:
    year = extract_year(os.path.basename(file_path))
    dataset = gdal.Open(file_path)
    array = dataset.GetRasterBand(1).ReadAsArray()
    array_celsius = kelvin_to_celsius(array)
    mean_temp = np.nanmean(array_celsius)
    years.append(year)
    mean_temperatures.append(mean_temp)

# Calculate the overall mean temperature
overall_mean = np.mean([temp for temp in mean_temperatures if temp is not None])

# Output folder for temporary images
output_folder = os.path.join(folder_path, "timeseries_images_labels")
os.makedirs(output_folder, exist_ok=True)

# Create each frame for the GIF
frames = []
durations = []
for i in range(1, len(years) + 1):
    fig, ax = plt.subplots(figsize=(10, 6))
    for j in range(i):
        color = 'red' if mean_temperatures[j] > overall_mean else 'blue'
        ax.plot(years[j], mean_temperatures[j], 'o', color=color)
        ax.annotate(f'{mean_temperatures[j]:.2f}', (years[j], mean_temperatures[j]), 
                    textcoords="offset points", xytext=(0,10), ha='center', 
                    bbox=dict(boxstyle="round,pad=0.3", fc=color, alpha=0.5))
    
    # Connect all points up to the current year with a line
    ax.plot(years[:i], mean_temperatures[:i], '-k', alpha=0.5)
    # Include label for the mean temperature line
    ax.axhline(y=overall_mean, color='gray', linestyle='--', label='Temperatura Promedio')
    
    ax.set_title("Temperatura Anual Promedio 1981-2023\nREGIÓN COSTA")
    ax.set_xlabel("Año")
    ax.set_ylabel("Temperatura Promedio (°C)")
    ax.set_xlim(1981, 2023)
    plt.tight_layout()
    
    # Display the legend in the lower right corner
    ax.legend(loc='lower right')
    
    frame_path = os.path.join(output_folder, f"frame_{i}.png")
    plt.savefig(frame_path)
    plt.close(fig)
    
    frames.append(imageio.imread(frame_path))
    durations.append(0.5)

# Adjust the duration of the last frame to 5 seconds
durations[-1] = 5

# Generate GIF with specified durations for each frame
gif_path = os.path.join(folder_path, "temperature_timeseries_labels.gif")
imageio.mimsave(gif_path, frames, duration=durations)

print(f"GIF created at {gif_path}")

C:\Users\carsk\AppData\Local\Temp\ipykernel_36100\2453135208.py:78: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frames.append(imageio.imread(frame_path))


GIF created at C:\Users\carsk\OneDrive - KU Leuven\Personal_projects\Temperature_map_EC\costa\temperature_timeseries_labels.gif


In [23]:
import os
import glob
import numpy as np
import pandas as pd
from osgeo import gdal

def kelvin_to_celsius(kelvin_array):
    """Convert temperature from Kelvin to Celsius, ignoring values < 250 K."""
    celsius_array = np.where(kelvin_array < 250, np.nan, kelvin_array - 273.15)
    return celsius_array

def extract_year(file_name):
    """Extract year from the file name."""
    return int(file_name.split('_')[-1].split('.')[0])

# Update this to the path where your TIFF files are located
folder_path = "C:\\Users\\carsk\\OneDrive - KU Leuven\\Personal_projects\\Temperature_map_EC\\era_YEARLY_Average"

# Pattern to match TIFF files
pattern = os.path.join(folder_path, '*.tif')
file_paths = sorted(glob.glob(pattern), key=lambda x: extract_year(x))

# Initialize lists to hold years and their mean temperatures
years = []
mean_temperatures = []

for file_path in file_paths:
    year = extract_year(os.path.basename(file_path))
    dataset = gdal.Open(file_path)
    array = dataset.GetRasterBand(1).ReadAsArray()
    array_celsius = kelvin_to_celsius(array)
    mean_temp = np.nanmean(array_celsius[array_celsius >= -273.15])  # Ignore invalid data
    years.append(year)
    mean_temperatures.append(mean_temp)

# Create a DataFrame from the lists
df_amazonia = pd.DataFrame({
    'date': years,
    'Amazonia': mean_temperatures
})

# Assuming you want to save this DataFrame as a CSV file:
output_csv_path = os.path.join(folder_path, "Nacional_temperatures.csv")
df_amazonia.to_csv(output_csv_path, index=False)

print(f"DataFrame of temperatures saved to {output_csv_path}")

DataFrame of temperatures saved to C:\Users\carsk\OneDrive - KU Leuven\Personal_projects\Temperature_map_EC\era_YEARLY_Average\Nacional_temperatures.csv


In [7]:
from PIL import Image
import glob
import os

def merge_images(map_image_path, time_series_image_path, output_path):
    # Load the map and time series images
    map_img = Image.open(map_image_path)
    time_series_img = Image.open(time_series_image_path)
    
    # Calculate dimensions for the merged image
    total_width = max(map_img.width, time_series_img.width)
    total_height = map_img.height + time_series_img.height
    
    # Create a new image with the combined dimensions
    merged_img = Image.new('RGB', (total_width, total_height), "white")  # Background color
    
    # Calculate offsets for centering
    map_offset = (total_width - map_img.width) // 2
    time_series_offset = (total_width - time_series_img.width) // 2
    
    # Paste the map and time series images into the merged image using the offsets
    merged_img.paste(map_img, (map_offset, 0))
    merged_img.paste(time_series_img, (time_series_offset, map_img.height))
    
    # Save the merged image
    merged_img.save(output_path)

maps_folder = "C:/Users/carsk/OneDrive - KU Leuven/Personal_projects/Temperature_map_EC/Amazonia/temp_images"
timeseries_folder = "C:/Users/carsk/OneDrive - KU Leuven/Personal_projects/Temperature_map_EC/Amazonia/timeseries_images_labels"
merged_folder = "C:/Users/carsk/OneDrive - KU Leuven/Personal_projects/Temperature_map_EC/Amazonia/merged_images"
os.makedirs(merged_folder, exist_ok=True)

# Assuming the map images are named like 'YYYY.png'
map_images = sorted(glob.glob(os.path.join(maps_folder, '*.png')))
map_years = [os.path.basename(img_path).split('.')[0] for img_path in map_images]

# For each map image, find the corresponding time series image by index and merge
for index, year in enumerate(map_years):
    map_image_path = os.path.join(maps_folder, f"{year}.png")
    time_series_image_path = os.path.join(timeseries_folder, f"frame_{index + 1}.png")
    output_path = os.path.join(merged_folder, f"merged_{year}.png")
    merge_images(map_image_path, time_series_image_path, output_path)

print("All images have been merged successfully.")

All images have been merged successfully.


In [8]:
merged_image_paths = sorted(glob.glob(os.path.join(merged_folder, '*.png')), key=lambda x: int(os.path.basename(x).split('_')[1].split('.')[0]))

final_gif_path = os.path.join(folder_path, "final_merged_animation.gif")
with imageio.get_writer(final_gif_path, mode='I', duration=0.5) as writer:
    for img_path in merged_image_paths:
        image = imageio.imread(img_path)
        writer.append_data(image)

print(f"Final merged GIF created at {final_gif_path}")

C:\Users\carsk\AppData\Local\Temp\ipykernel_36100\2293174356.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path)


Final merged GIF created at C:\Users\carsk\OneDrive - KU Leuven\Personal_projects\Temperature_map_EC\Amazonia\final_merged_animation.gif
